# Extracting radar values at rain gauge locations (time series)

The previous notebook showed the workflow of extracting the radar values at gauge locations for a single scan. In this notebook, snippets from the previous notebook will be re-used to extract the values for the entire case study period.

In [1]:
import pyart
import wradlib as wrl
import pandas as pd
import tempfile
import os
import numpy as np

import pytz
import datetime as dt

from copy import deepcopy
import cartopy.crs as ccrs
import cartopy.io.shapereader as shpreader
import cartopy.feature as cfeature

import boto3
from botocore.handlers import disable_signing


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119



C:\Users\irene\Anaconda3\envs\saveurproj\lib\site-packages\pyart\graph\cm.py:104: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if 'red' in spec:
C:\Users\irene\Anaconda3\envs\saveurproj\lib\site-packages\pyart\graph\cm_colorblind.py:32: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if 'red' in spec:
C:\Users\irene\Anaconda3\envs\saveurproj\lib\site-packages\botocore\vendored\requests\packages\urllib3\_collections.py:1: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Mapping, MutableMapping


In [2]:
def rounder(t):
    """
    Rounds the time to the nearest hour.
    """
    if t.minute >= 30:
        return t.replace(second=0, microsecond=0, minute=0, hour=t.hour+1)
    else:
        return t.replace(second=0, microsecond=0, minute=0)

In [3]:
def get_radar_scan(station='KLOT', date=None, key_index=-20):
    
    '''
    Function will pull the latest radar scan from any radar site using 
    Amazon S3.
    ----------
    Station = Four letter NEXRAD identifier
              Example: 'KEPZ'
    Date = default is none for current date, else enter date in format "YYYY/MM/DD"
    Ex: date ='2013/11/17
    Key_index = Number of keys you want pulled from most recent scan.
    Ex: key_index = -15 would pull ht most recent 15 scans
    '''
    
    # Creating a bucket and a client to be able to pull data from AWS and setting it as unsigned
    bucket = 'noaa-nexrad-level2'
    s3 = boto3.resource('s3')
    s3.meta.client.meta.events.register('choose-signer.s3.*', disable_signing)
    
    # Connects the bucket create above with radar data
    aws_radar = s3.Bucket(bucket)
    
    # Setting the date and time to current...
    # This will allow for allow the current date's radar scands to be pulled
    if date == None:
        target_string = datetime.datetime.utcnow().strftime('%Y/%m/%d/'+station)
    else:
        target_string = date+'/'+station
    
    for obj in aws_radar.objects.filter(Prefix= target_string):
        '{0}:{1}'.format(aws_radar.name, obj.key)
    my_list_of_keys = [this_object.key for this_object in aws_radar.objects.filter(Prefix= target_string)]
    keys = my_list_of_keys[key_index:]
    newkeys = []
    for key in keys:
        if 'MDM' in key:
            pass
        elif key.endswith('.tar'):
            pass
        else:
            newkeys.append(key)
    #print(newkeys)
    return aws_radar, newkeys

Get the filenames of the available files for download, for the specified radar `station` (KLOT for Chicago), `date`, and `key_index`. `key_index` refers to the N most recent files.

In [4]:
# Setting radar, date of radar scans needed, and key index (amount of items in list)
aws_radar, keys = get_radar_scan(station='KLOT', date='2013/04/17', key_index=-400) 

Set the filenames to be opened.

In [5]:
# load CCN gauge locations
CCN_gauge_locations_fname = 'C:/Users/irene/Documents/Work/Data/Cook_County/CookCounty_gage_locations.csv'
# load CCN gauge observations
CCN_gauge_observations_fname = 'C:/Users/irene/Documents/Work/Data/Cook_County/WaterYear2013.csv'

Read.

In [6]:
df_gauge_loc = pd.read_csv(CCN_gauge_locations_fname,header=0)
df_gauge = pd.read_csv(CCN_gauge_observations_fname,header=0)

Setting up variables that are reccuringly used.

In [7]:
# set the timezone
timezone = pytz.timezone("America/Chicago")

In [8]:
# gauge locations
x = df_gauge_loc['Longitude - West'].values
y = df_gauge_loc['Latitude'].values

In [9]:
df_gauge['Datetime'] = pd.to_datetime(df_gauge['Date/Time'])
df_gauge['Datetime'] = df_gauge['Datetime'].dt.tz_localize(timezone,ambiguous='NaT',nonexistent ='NaT')

In [10]:
proj = wrl.georef.epsg_to_osr(4326)

In [11]:
keys

['2013/04/17/KLOT/KLOT20130417_000444_V06.gz',
 '2013/04/17/KLOT/KLOT20130417_001029_V06.gz',
 '2013/04/17/KLOT/KLOT20130417_001615_V06.gz',
 '2013/04/17/KLOT/KLOT20130417_002201_V06.gz',
 '2013/04/17/KLOT/KLOT20130417_002746_V06.gz',
 '2013/04/17/KLOT/KLOT20130417_003331_V06.gz',
 '2013/04/17/KLOT/KLOT20130417_003916_V06.gz',
 '2013/04/17/KLOT/KLOT20130417_004500_V06.gz',
 '2013/04/17/KLOT/KLOT20130417_005048_V06.gz',
 '2013/04/17/KLOT/KLOT20130417_005633_V06.gz',
 '2013/04/17/KLOT/KLOT20130417_010220_V06.gz',
 '2013/04/17/KLOT/KLOT20130417_010806_V06.gz',
 '2013/04/17/KLOT/KLOT20130417_011350_V06.gz',
 '2013/04/17/KLOT/KLOT20130417_012138_V06.gz',
 '2013/04/17/KLOT/KLOT20130417_012724_V06.gz',
 '2013/04/17/KLOT/KLOT20130417_013311_V06.gz',
 '2013/04/17/KLOT/KLOT20130417_013856_V06.gz',
 '2013/04/17/KLOT/KLOT20130417_014442_V06.gz',
 '2013/04/17/KLOT/KLOT20130417_015026_V06.gz',
 '2013/04/17/KLOT/KLOT20130417_015613_V06.gz',
 '2013/04/17/KLOT/KLOT20130417_020158_V06.gz',
 '2013/04/17/

In [13]:
def get_radar_time_from_fname(fname):
    fn = fname.rsplit('/',1)[-1].strip('.gz')
    dtime_utc = dt.datetime.strptime(fn,'KLOT%Y%m%d_%H%M%S_V06')
    dtime_utc = pytz.utc.localize(dtime_utc)
    return dtime_utc

In [14]:
dtimes = []
for key in keys:
    dtimes.append(get_radar_time_from_fname(key))

In [15]:
np.mean(np.diff(dtimes))

datetime.timedelta(seconds=302, microseconds=84211)

In [16]:
newkeys=keys

Read one radar file once to get the radar parameters and setup the polar neighbor object. This takes some time to generate, and it makes most sense to generate this once at the start and just reuse the object.

In [17]:
# loop through the keys by iterating nframe
nframe = 14

# open a temporary local file
localfile = tempfile.NamedTemporaryFile(delete=False)
localfile_name = localfile.name
localfile.close()

# download to temporary file and read to radar object using pyart
aws_radar.download_file(newkeys[nframe], localfile_name)
radar = pyart.io.read(localfile_name)

# delete temporary file to save space
os.remove(localfile_name)

In [18]:
radar_slice0 = radar.get_slice(0)

In [19]:
sitecoords = (radar.longitude['data'][0],radar.latitude['data'][0])
az = radar.azimuth['data'][radar_slice0]
r = radar.range['data']

# create the polar neighbor object
polarneighbs = wrl.verify.PolarNeighbours(r, az, sitecoords, proj, x,y, nnear=9)

C:\Users\irene\Anaconda3\envs\saveurproj\lib\site-packages\wradlib\georef\polar.py:503: UserWarning: The azimuth angles of the current dataset are not equidistant.
  "dataset are not equidistant.", UserWarning)


In [20]:
# create empty dataframe where we will put the extracted values
columns=['DateTime','G1','G2','G3','G4','G5','G6','G7','G8',
                           'G9','G10','G11','G12','G13','G14','G15','G16','G17',
                           'G18','G19','G20','G21','G22','G23','G24','G25']
df = pd.DataFrame(columns=columns)

In [21]:
for nframe in range(len(newkeys)):
    print(newkeys[nframe])
    
    # 1. Read and open file
    # open a temporary local file
    localfile = tempfile.NamedTemporaryFile(delete=False)
    localfile_name = localfile.name
    localfile.close()

    # download to temporary file and read to radar object using pyart
    aws_radar.download_file(newkeys[nframe], localfile_name)
    radar = pyart.io.read(localfile_name)

    # delete temporary file to save space
    os.remove(localfile_name)
    
    # get local time of radar
    fname = newkeys[nframe].rsplit('/',1)[-1].strip('.gz')
    dtime_utc = dt.datetime.strptime(fname,'KLOT%Y%m%d_%H%M%S_V06')
    dtime_utc = pytz.utc.localize(dtime_utc)
    
    # 2. Convert reflectivity to rain rate
    gatefilter = pyart.filters.GateFilter(radar)
    # Develop your gatefilter first
    # exclude masked gates from the gridding
    #gatefilter = pyart.filters.GateFilter(radar)
    gatefilter.exclude_transition()
    gatefilter.exclude_masked('reflectivity')
    # Mask reflectivity
    radar.fields["corrected_reflectivity"] = deepcopy(radar.fields["reflectivity"])
    radar.fields["corrected_reflectivity"]["data"] = np.ma.masked_where(
        gatefilter._gate_excluded, radar.fields["corrected_reflectivity"]["data"])
    rr = pyart.retrieve.est_rain_rate_z(radar, refl_field="corrected_reflectivity")
    
    radar.add_field('rainrate',rr,replace_existing=True)
    
    # Mask out last 10 gates of each ray, this removes the "ring" around the radar.
    radar.fields['rainrate']['data'][:, -10:] = np.ma.masked
    
    # 3.2 Get radar data
    # Get slice
    radar_slice0 = radar.get_slice(0)
    rr_0 = radar.fields['rainrate']['data'][radar_slice0, :]

    # get radar values of 9 nearest neighbors
    radar_at_gages = polarneighbs.extract(rr_0)
    # get mean
    radar_at_gages_mean = np.mean(radar_at_gages,axis=1)
    # convert rain rate to rain amount
    radar_at_gages_amount = wrl.trafo.r_to_depth(radar_at_gages_mean,interval=256)
    
    df_ = pd.DataFrame(data=[radar_at_gages_amount],columns=columns[1:])
    df_['DateTime(UTC)'] = dtime_utc
    
    df = df.append(df_)

2013/04/17/KLOT/KLOT20130417_000444_V06.gz


C:\Users\irene\Anaconda3\envs\saveurproj\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


2013/04/17/KLOT/KLOT20130417_001029_V06.gz
2013/04/17/KLOT/KLOT20130417_001615_V06.gz
2013/04/17/KLOT/KLOT20130417_002201_V06.gz
2013/04/17/KLOT/KLOT20130417_002746_V06.gz
2013/04/17/KLOT/KLOT20130417_003331_V06.gz
2013/04/17/KLOT/KLOT20130417_003916_V06.gz
2013/04/17/KLOT/KLOT20130417_004500_V06.gz
2013/04/17/KLOT/KLOT20130417_005048_V06.gz
2013/04/17/KLOT/KLOT20130417_005633_V06.gz
2013/04/17/KLOT/KLOT20130417_010220_V06.gz
2013/04/17/KLOT/KLOT20130417_010806_V06.gz
2013/04/17/KLOT/KLOT20130417_011350_V06.gz
2013/04/17/KLOT/KLOT20130417_012138_V06.gz
2013/04/17/KLOT/KLOT20130417_012724_V06.gz
2013/04/17/KLOT/KLOT20130417_013311_V06.gz
2013/04/17/KLOT/KLOT20130417_013856_V06.gz
2013/04/17/KLOT/KLOT20130417_014442_V06.gz
2013/04/17/KLOT/KLOT20130417_015026_V06.gz
2013/04/17/KLOT/KLOT20130417_015613_V06.gz
2013/04/17/KLOT/KLOT20130417_020158_V06.gz
2013/04/17/KLOT/KLOT20130417_020744_V06.gz
2013/04/17/KLOT/KLOT20130417_021331_V06.gz
2013/04/17/KLOT/KLOT20130417_021916_V06.gz
2013/04/17/

2013/04/17/KLOT/KLOT20130417_170655_V06.gz
2013/04/17/KLOT/KLOT20130417_171133_V06.gz
2013/04/17/KLOT/KLOT20130417_171612_V06.gz
2013/04/17/KLOT/KLOT20130417_172049_V06.gz
2013/04/17/KLOT/KLOT20130417_172735_V06.gz
2013/04/17/KLOT/KLOT20130417_173213_V06.gz
2013/04/17/KLOT/KLOT20130417_173652_V06.gz
2013/04/17/KLOT/KLOT20130417_174130_V06.gz
2013/04/17/KLOT/KLOT20130417_174609_V06.gz
2013/04/17/KLOT/KLOT20130417_175048_V06.gz
2013/04/17/KLOT/KLOT20130417_175526_V06.gz
2013/04/17/KLOT/KLOT20130417_180005_V06.gz
2013/04/17/KLOT/KLOT20130417_180444_V06.gz
2013/04/17/KLOT/KLOT20130417_180923_V06.gz
2013/04/17/KLOT/KLOT20130417_181401_V06.gz
2013/04/17/KLOT/KLOT20130417_181840_V06.gz
2013/04/17/KLOT/KLOT20130417_182317_V06.gz
2013/04/17/KLOT/KLOT20130417_182756_V06.gz
2013/04/17/KLOT/KLOT20130417_183234_V06.gz
2013/04/17/KLOT/KLOT20130417_183714_V06.gz
2013/04/17/KLOT/KLOT20130417_184152_V06.gz
2013/04/17/KLOT/KLOT20130417_184629_V06.gz
2013/04/17/KLOT/KLOT20130417_185106_V06.gz
2013/04/17/

In [22]:
df['Datetime'] = df['DateTime(UTC)'].dt.tz_convert('US/Central')

In [23]:
df.to_csv('KLOT_20130417_RainAmounts_at_CookCountyGauges.csv')